<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamB/blob/main/Building_Staging_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing csv Data into a staging Database

Importing required libruaries

In [84]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv

retiriving csv Files form GitHub DatamanagementAndArchiving

In [85]:
!git clone https://github.com/Fuenfgeld/DatamanagementAndArchiving

fatal: destination path 'DatamanagementAndArchiving' already exists and is not an empty directory.


Mounting Google Drive with Shared Team Folder DMA_2023_D

Authentification

In [86]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining File Paths and Relevant Folders

DB_FILE_PATH -->  where source database will reside

In [87]:
patient_type = "covid19"

# Path to all data files in SynteticPatientData
material_path = "/content/DatamanagementAndArchiving/SynteticPatientData"

# Source database path
DB_FILE_PATH = f"/content/drive/Shareddrives/DMA_2023_D/DMA2023TeamB/source_dbs/Source_{patient_type}_Staging.db"

# folder contains csv files
csv_path = f"{material_path}/{patient_type}/"

SQL queries to create tables

In [88]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}

In [89]:
# sql query to create disease table
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""
#  sql query to create patients table
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
# sql query to create encounters table
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                       );"""
# sql query to create careplans table
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
# sql query to create conditions table
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
# sql query to create medications table
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           DISPENSES INTEGER,
                           TOTALCOST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           
                       );"""
# sql query to create procedures table
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""


# sql query to create observations table
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

# sql query to create devices table
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""

# sql query to create Imaging_studies table
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

# sql query to create Immunizations table
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

Import csv files into database tables

In [90]:
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()


def insert_values_to_table(cursor, table_name, csv_file_path):

    """
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    :param table_name: table name in the database to insert the data into
    :param csv_file_path: path of the csv file to process
    :return: None
    """

    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')


def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table
    :param sql_cursor: sqlite cursor
    :param table_name: table name to get the column names from
    :return: a comma separated string with column names, an integer with number of columns
    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count




In [91]:
# import csv into db
conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          csv_file = csv_path + name + ".csv"
          #print(name, csv_file)
          
          cursor.execute(sql_table[name])
          cursor.execute(f"""DELETE FROM {name}""")
          insert_values_to_table(cursor, name, csv_file)
          conn.commit()
else:
        print('Connection to database failed')


Nothing to insert
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
Nothing to insert
SQL insert process finished


In [92]:
conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_CONDITIONS (
                            CONDITIONS_CODE STRING PRIMARY KEY,
                            CONDITIONS_DESCRIPTION STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_DEVICES (
                            DEVICES_CODE STRING PRIMARY KEY,
                            DEVICES_DESCRIPTION STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_MEDICATIONS (
                            MEDICATIONS_CODE STRING PRIMARY KEY,
                            MEDICATIONS_DESCRIPTION STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_IMMUNIZATIONS (
                            IMMUNIZATIONS_CODE STRING PRIMARY KEY,
                            IMMUNIZATIONS_DESCRIPTION STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_ENCOUNTERS (
                            ENCOUNTERS_ID STRING PRIMARY KEY,
                            ENCOUNTERCLASS STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_PROCEDURES (
                            PROCEDURES_CODE STRING PRIMARY KEY,
                            PROCEDURES_DESCRIPTION STRING
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS S_PATIENTS (
                            PATIENTS_ID STRING PRIMARY KEY,
                            PATIENTS_BIRTHDATE DATE,
                            PATIENTS_DEATHDATE DATE,
                            PATIENTS_GENDER STRING
                           )"""
conn.execute(query)
conn.commit()

In [93]:
conn = connect_to_db(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
list_of_tables = cursor.fetchall()
list_of_tables_in_database = []
list_of_s_tables=[]
for tupel in list_of_tables:
  list_of_tables_in_database.append(tupel[0])
for table in list_of_tables_in_database:
  if table[0]=='S' and table[1]== '_':
    list_of_s_tables.append(table)

for table in list_of_s_tables:
  conn = sqlite3.connect(DB_FILE_PATH)
  query = f"""DELETE FROM {table}"""
  conn.execute(query)
  conn.commit()

In [94]:
## List of tables in database 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('disease',), ('patients',), ('encounters',), ('careplans',), ('conditions',), ('medications',), ('procedures',), ('observations',), ('devices',), ('imaging_studies',), ('immunizations',), ('S_CONDITIONS',), ('S_DEVICES',), ('S_MEDICATIONS',), ('S_IMMUNIZATIONS',), ('S_ENCOUNTERS',), ('S_PROCEDURES',), ('S_PATIENTS',)]


In [95]:
## check data in a table
cursor.execute("SELECT count(*) FROM immunizations;")
rows = cursor.fetchall()
for row in rows:
    print(row)

(13125,)


commit Changes to Staging Database

In [96]:
conn.commit()

Close connection to Staging Database

In [97]:
conn.close()

Creating Staging (S_CONDITIONS) Tables

In [98]:
import sqlite3
from pandas import DataFrame

conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT CONDITIONS.CODE, CONDITIONS.DESCRIPTION FROM CONDITIONS')
DF_S_Conditions = DataFrame(cursor.fetchall())
DF_S_Conditions.columns = ['CONDITIONS_CODE', 'CONDITIONS_DESCRIPTION']
conn.close()

In [99]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_Conditions.to_sql('S_CONDITIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_DEVICES) Tables

In [100]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT DEVICES.CODE, DEVICES.DESCRIPTION FROM DEVICES')
DF_S_DEVICES = DataFrame(cursor.fetchall())
DF_S_DEVICES.columns = ['DEVICES_CODE', 'DEVICES_DESCRIPTION']
conn.close()

In [101]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_DEVICES.to_sql('S_DEVICES',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_MEDICATIONS) Tables

In [102]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT MEDICATIONS.CODE, MEDICATIONS.DESCRIPTION FROM MEDICATIONS')
DF_S_MEDICATIONS = DataFrame(cursor.fetchall())
DF_S_MEDICATIONS.columns = ['MEDICATIONS_CODE', 'MEDICATIONS_DESCRIPTION']
conn.close()

In [103]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_MEDICATIONS.to_sql('S_MEDICATIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_IMMUNIZATIONS) Tables


In [104]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT IMMUNIZATIONS.CODE, IMMUNIZATIONS.DESCRIPTION FROM IMMUNIZATIONS')
DF_S_IMMUNIZATIONS = DataFrame(cursor.fetchall())
DF_S_IMMUNIZATIONS.columns = ['IMMUNIZATIONS_CODE', 'IMMUNIZATIONS_DESCRIPTION']
conn.close()

In [105]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_IMMUNIZATIONS.to_sql('S_IMMUNIZATIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_ENCOUNTERS) Tables

In [106]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT ENCOUNTERS.ID, ENCOUNTERS.ENCOUNTERCLASS FROM ENCOUNTERS')
DF_S_ENCOUNTERS = DataFrame(cursor.fetchall())
DF_S_ENCOUNTERS.columns = ['ENCOUNTERS_ID', 'ENCOUNTERCLASS']
conn.close()

In [107]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_ENCOUNTERS.to_sql('S_ENCOUNTERS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_PROCEDURES) Tables

In [108]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT PROCEDURES.CODE, PROCEDURES.DESCRIPTION FROM PROCEDURES')
DF_S_PROCEDURES = DataFrame(cursor.fetchall())
DF_S_PROCEDURES.columns = ['PROCEDURES_CODE', 'PROCEDURES_DESCRIPTION']
conn.close()

In [109]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_PROCEDURES.to_sql('S_PROCEDURES',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

Creating Staging (S_PATIENTS) Tables

In [110]:
conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT PATIENTS.ID, PATIENTS.BIRTHDATE, PATIENTS.DEATHDATE, PATIENTS.GENDER FROM PATIENTS')
DF_S_PATIENTS = DataFrame(cursor.fetchall())
DF_S_PATIENTS.columns = ['PATIENTS_ID', 'PATIENTS_BIRTHDATE', 'PATIENTS_DEATHDATE', 'PATIENTS_GENDER']
conn.close()

In [111]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_S_PATIENTS.to_sql('S_PATIENTS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

TEST CODE

In [112]:
# conn = sqlite3.connect(DB_FILE_PATH)
# cursor = conn.cursor()
# #cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# #cursor.execute("SELECT count(*) FROM conditions")
# print(cursor.fetchall())

Building Staging Factstable

In [113]:
conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_CONDITIONS (
                            CONDITIONS_CODE STRING,
                            CONDITIONS_DESCRIPTION STRING,
                            CONDITIONS_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_DEVICES (
                            DEVICES_CODE STRING,
                            DEVICES_DESCRIPTION STRING,
                            DEVICES_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_MEDICATIONS (
                            MEDICATIONS_CODE STRING,
                            MEDICATIONS_DESCRIPTION STRING,
                            MEDICATIONS_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_IMMUNIZATIONS (
                            IMMUNIZATIONS_CODE STRING,
                            IMMUNIZATIONS_DESCRIPTION STRING,
                            IMMUNIZATIONS_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_ENCOUNTERS (
                            ENCOUNTERS_ID STRING PRIMARY KEY,
                            ENCOUNTERCLASS STRING,
                            ENCOUNTERS_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_PROCEDURES (
                            PROCEDURES_CODE STRING,
                            PROCEDURES_DESCRIPTION STRING,
                            PROCEDURES_PATIENT_ID
                           )"""
conn.execute(query)
conn.commit()

conn = sqlite3.connect(DB_FILE_PATH)
query = """CREATE TABLE IF NOT EXISTS SSS_PATIENTS (
                            PATIENTS_ID STRING PRIMARY KEY,
                            PATIENTS_BIRTHDATE DATE,
                            PATIENTS_DEATHDATE DATE,
                            PATIENTS_GENDER STRING
                           )"""
conn.execute(query)
conn.commit()

In [114]:
conn = connect_to_db(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
list_of_tables = cursor.fetchall()
list_of_tables_in_database = []
list_of_ss_tables=[]
for tupel in list_of_tables:
  list_of_tables_in_database.append(tupel[0])
for table in list_of_tables_in_database:
  if table[0]=='S' and table[1]== 'S':
    list_of_ss_tables.append(table)

for table in list_of_ss_tables:
  conn = sqlite3.connect(DB_FILE_PATH)
  query = f"""DELETE FROM {table}"""
  conn.execute(query)
  conn.commit()

In [115]:
import sqlite3
from pandas import DataFrame

conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT CONDITIONS.CODE, CONDITIONS.DESCRIPTION, CONDITIONS.PATIENT FROM CONDITIONS')
DF_SSS_Conditions = DataFrame(cursor.fetchall())
DF_SSS_Conditions.columns = ['CONDITIONS_CODE', 'CONDITIONS_DESCRIPTION', 'CONDITIONS_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT DEVICES.CODE, DEVICES.DESCRIPTION, DEVICES.PATIENT FROM DEVICES')
DF_SSS_DEVICES = DataFrame(cursor.fetchall())
DF_SSS_DEVICES.columns = ['DEVICES_CODE', 'DEVICES_DESCRIPTION', 'DEVICES_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT MEDICATIONS.CODE, MEDICATIONS.DESCRIPTION, MEDICATIONS.PATIENT FROM MEDICATIONS')
DF_SSS_MEDICATIONS = DataFrame(cursor.fetchall())
DF_SSS_MEDICATIONS.columns = ['MEDICATIONS_CODE', 'MEDICATIONS_DESCRIPTION', 'MEDICATIONS_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT IMMUNIZATIONS.CODE, IMMUNIZATIONS.DESCRIPTION, IMMUNIZATIONS.PATIENT FROM IMMUNIZATIONS')
DF_SSS_IMMUNIZATIONS = DataFrame(cursor.fetchall())
DF_SSS_IMMUNIZATIONS.columns = ['IMMUNIZATIONS_CODE', 'IMMUNIZATIONS_DESCRIPTION', 'IMMUNIZATIONS_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT ENCOUNTERS.ID, ENCOUNTERS.ENCOUNTERCLASS, ENCOUNTERS.PATIENT FROM ENCOUNTERS')
DF_SSS_ENCOUNTERS = DataFrame(cursor.fetchall())
DF_SSS_ENCOUNTERS.columns = ['ENCOUNTERS_ID', 'ENCOUNTERCLASS', 'ENCOUNTERS_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT PROCEDURES.CODE, PROCEDURES.DESCRIPTION, PROCEDURES.PATIENT FROM PROCEDURES')
DF_SSS_PROCEDURES = DataFrame(cursor.fetchall())
DF_SSS_PROCEDURES.columns = ['PROCEDURES_CODE', 'PROCEDURES_DESCRIPTION', 'PROCEDURES_PATIENT_ID']
conn.close()


conn = sqlite3.connect(DB_FILE_PATH)
cursor = conn.cursor()
cursor.execute ('SELECT PATIENTS.ID, PATIENTS.BIRTHDATE, PATIENTS.DEATHDATE, PATIENTS.GENDER FROM PATIENTS')
DF_SSS_PATIENTS = DataFrame(cursor.fetchall())
DF_SSS_PATIENTS.columns = ['PATIENTS_ID', 'PATIENTS_BIRTHDATE', 'PATIENTS_DEATHDATE', 'PATIENTS_GENDER']
conn.close()


In [116]:
conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_Conditions.to_sql('SSS_CONDITIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_DEVICES.to_sql('SSS_DEVICES',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_MEDICATIONS.to_sql('SSS_MEDICATIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_IMMUNIZATIONS.to_sql('SSS_IMMUNIZATIONS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_ENCOUNTERS.to_sql('SSS_ENCOUNTERS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_PROCEDURES.to_sql('SSS_PROCEDURES',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

conn = sqlite3.connect(DB_FILE_PATH)
DF_SSS_PATIENTS.to_sql('SSS_PATIENTS',conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()
